# Step 1. Reload files & Import Modules - 匯入模組

## 1-1. Reload files - 重讀檔案

In [1]:
# reload file if you eaited them
import crawler104, config.search_params, async_example, threaded_async_job, jobs104
import data_lake
import importlib
importlib.reload(crawler104)
importlib.reload(async_example)
importlib.reload(threaded_async_job)
importlib.reload(jobs104)
importlib.reload(config.search_params)
importlib.reload(data_lake)

<module 'data_lake' from 'C:\\Users\\Rekam\\Documents\\Jobscan\\data_lake.py'>

## 1-2. Import Modules - 匯入模組

In [2]:
# import library
from crawler104 import Crawler104
from config.search_params import get_filter_params
from data_lake import DataLake

In [1]:
# 快速執行 reload & import
%run main.py

# Step 2. Data Source - 爬蟲抓資料

## 2-1. Web Crawler - 抓取Jobs清單

In [2]:
# custom filter params for search - for yidti
role = {'ro':'全職'}
keyword = {'keyword':"後端工程師 python"}
# area = {'area':['新北市', '台北市', '桃園市', '台中市']}
isnew = {'isnew':'三日內'}
jobexp = {'jobexp':['1年以下', '1-3年']}
# 預設
mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
order = {'order':'日期排序'}
asc = {'asc':'遞減'}
# filter_params = get_filter_params(role, keyword, area, isnew, jobexp, mode, order, asc)
filter_params = get_filter_params(role, keyword, isnew, jobexp, mode, order, asc)

# keywords for filter job again
job_keywords = ('工程','資料','python','data','數據','後端')
# Exclude keywords to filter out companies related to gambling or others that I don't want to consider.
company_exclude = ('新加坡商冕創有限公司','新博軟體開發股份有限公司','現觀科技股份有限公司'
                   ,'全富數位有限公司','杰思數位有限公司','博凡星國際有限公司',
                  '尊博科技股份有限公司','新騎資訊有限公司','新加坡商鈦坦科技股份有限公司台灣分公司',
                   '豪穎科技股份有限公司','塶樂微創有限公司','磐弈有限公司',
                   '聯訊網路有限公司','冶金數位科技有限公司','肥貓科技有限公司',
                   '無名科技有限公司',
                  )
user = "yidti"
crawler = Crawler104(filter_params, user)
crawler.run(job_keywords, company_exclude)

url: https://www.104.com.tw/jobs/search/?ro=1&keyword=%E5%BE%8C%E7%AB%AF%E5%B7%A5%E7%A8%8B%E5%B8%AB+python&isnew=3&jobexp=1%2C3&mode=l&order=16&asc=0


Loading: 100%|██████████| 81/81 [01:48<00:00,  1.34s/page]


載入81頁 | 載入2406筆資料 | 過濾剩1810筆資料 | 花費 125.73 秒


## 2-2. Web Scraper - 抓取Jobs內容(異步&多線程)

In [3]:
crawler.detail()

Processing jobs: 100%|██████████| 1810/1810 [05:11<00:00,  5.82job/s]

Scraping Details for 1810 Jobs | 花費 311.1 秒


## 2-3. Export Flie - 輸出至Excel

In [4]:
# output to excel file (job)
crawler.export_excel()

CSV文件保存成功: output/yidti_2024-02-07_1.xlsx


# Step 3. Data Lake - 資料存入NoSQL

In [75]:
# ouput to noSQL (job, company, industry)
data_Lake = DataLake()
data_Lake.export_nosql(user, crawler)
data_Lake.filter(job_keywords, company_exclude)

Update 1800 records, Insert 10 records in jobs_104 collection
job keywords - 已刪除不符合關鍵字的文件數量: 0
company exclude - 已刪除符合條件的文件數量: 10


# Step 4 - Data Warehouse - 資料存入MySQL

In [74]:
# 快速執行 reload & import
%run main.py

In [65]:

job_keywords = ('工程','資料','python','data','數據','後端')
company_exclude = ('新加坡商冕創有限公司','新博軟體開發股份有限公司','現觀科技股份有限公司'
                   ,'全富數位有限公司','杰思數位有限公司','博凡星國際有限公司',
                  '尊博科技股份有限公司','新騎資訊有限公司','新加坡商鈦坦科技股份有限公司台灣分公司',
                   '豪穎科技股份有限公司','塶樂微創有限公司','磐弈有限公司',
                   '聯訊網路有限公司','冶金數位科技有限公司','肥貓科技有限公司',
                   '無名科技有限公司',
                  )
def include_keywords(collection, job_keywords):
    # 構建正則表達式
    regex_pattern = '|'.join(job_keywords)
    
    # 刪除不符合關鍵字的文件
    delete_query = {'職缺': {'$not': {'$regex': regex_pattern, '$options': 'i'}}}
    delete_result = collection.delete_many(delete_query)
    print("已刪除不符合關鍵字的文件數量:", delete_result.deleted_count)


def exclude_keywords(collection, company_exclude):
    # 構建要刪除的文件的查詢條件
    delete_query = {'公司': {'$in': company_exclude}}
    # 刪除符合條件的文件
    delete_result = collection.delete_many(delete_query)
    print("已刪除符合條件的文件數量:", delete_result.deleted_count)




    
include_keywords(collection, job_keywords)
exclude_keywords(collection, company_exclude)



已刪除不符合關鍵字的文件數量: 0
已刪除符合條件的文件數量: 10


In [6]:
import pandas as pd
import pymongo

# data_Lake.load()
noSQL_DB_name = "job_db"
collection_name = "jobs_104"
df_jobs = pd.DataFrame()
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client[noSQL_DB_name]
collection = db[collection_name]


In [66]:
import pandas as pd

# latest_date_record = collection.find_one(
#             filter={},
#             sort=[("data stamp", -1)],  # 按照 "搜集資料日期" 字段降序排序
#             projection={"id": 0, "data stamp": 1}  # 只返回 "搜集資料日期" 字段，不返回 "_id" 字段
#         )
stamp = collection.find_one(
    filter= {}, # 返回集合中的所有文檔
    sort=[("data stamp", -1)], # 按照字段進行降序排序。 -1 表示降序排列，1 表示升序排列
    projection={"_id": 0, "data stamp": 1}) # 指定了返回的文檔中的欄位, 0 表示排除, 1代表返回
latest_stamp = stamp['data stamp']
data_list = list(collection.find({"data stamp": {"$eq": latest_stamp}})) #  $eq（equal to）操作符
if data_list:
    df_jobs = pd.DataFrame(data_list)
    df_jobs.set_index("id", inplace=True)  # 在這裡使用 set_index 方法將 id 設置為索引
    df_jobs = df_jobs.drop(["_id", "data stamp"], axis = 1)

df_jobs

,更新,職缺,職缺_link,公司_id,公司,公司_link,產業_id,產業,縣市,區域,...,其他,待遇,性質,管理,出差,時段,休假,可上,人數,福利
id,,,,,,,,,,,,,,,,,,,,,
11386098,2024/02/05,服務後端軟體工程師,https://www.104.com.tw/job/6s1ki,130000000142989,華捷智能股份有限公司,https://www.104.com.tw/company/1a2x6bl1n1,1001001002,電腦軟體服務業,台北市,內湖區,...,,待遇面議,全職,不需負擔管理責任,無需出差外派,日班,依公司規定,不限,1~2人,- 福利制度：\r\n正職人員福利\r\n【優於勞基法假勤制度】\r\n◆ 周休二日，彈性上...
13199708,2024/02/05,Database/Back-end Engineer 後端資料庫開發工程師,https://www.104.com.tw/job/7uwyk,130000000204211,易得雲端股份有限公司,https://www.104.com.tw/company/1a2x6bmcvn,1001001003,網際網路相關業,台北市,內湖區,...,,"年薪900,000~1,600,000元",全職,不需負擔管理責任,無需出差外派,日班,週休二日,一個月內,1~2人,"- 福利制度：【法定項目】\r\n勞健保, 退休金, 周休二日\r\n【其他福利】\r\n優..."
12478109,2024/02/05,C++ 後端軟體工程師 | Fintech,https://www.104.com.tw/job/7fg65,130000000165262,一通數位有限公司,https://www.104.com.tw/company/1a2x6blitq,1001001002,電腦軟體服務業,台北市,內湖區,...,- 其他：,"月薪60,000~80,000元",全職,管理人數未定,無需出差外派,日班，9-18或10-19,週休二日,一週內,不限,- 福利制度：◆設計明亮的工作空間以及舒適的員工休息區，整體寬敞舒心，讓夥伴們擁有輕鬆的上班...
13770804,2024/02/05,Wordpress網站開發工程師,https://www.104.com.tw/job/875mc,28936424000,軸心整合設計有限公司,https://www.104.com.tw/company/dajzznk,1006003001,廣告行銷公關業,新北市,永和區,...,,待遇面議,全職,不需負擔管理責任,無需出差外派,日班,週休二日,不限,1人,- 福利制度：- 不定時部門聚餐，慶生會，國內旅遊\r\n- 優於勞基法休假制度\r\n- ...
13768719,2024/02/05,應用軟體工程師,https://www.104.com.tw/job/8740f,24497254000,威潤科技股份有限公司,https://www.104.com.tw/company/b95165s,1001002003,通訊機械器材相關業,台北市,內湖區,...,- 其他：可提供相關實績作品集(Github鏈結亦可)參考\r\n1. 具清晰思慮且可獨立工...,待遇面議,全職,不需負擔管理責任,無需出差外派,日班,週休二日,不限,1人,- 福利制度：【獎金制度】\r\n．\t年終獎金\r\n．\t績效獎金\r\n．\t三節獎金...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12795553,2024/02/06,PHP資深網頁後端工程師,https://www.104.com.tw/job/7m941,130000000189365,核果智能科技股份有限公司,https://www.104.com.tw/company/1a2x6bm1f9,1001001006,其它軟體及網路相關業,台北市,松山區,...,,"年薪900,000~1,500,000元",全職,不需負擔管理責任,無需出差外派,日班,依公司規定,不限,1~3人,- 福利制度：核果智能科技(Walnutek) 辦公室位在捷運台北小巨蛋站旁的TTA(Tai...
13350820,2024/02/06,《台北》C# 後端工程師,https://www.104.com.tw/job/7y5k4,70613972000,泰偉電子股份有限公司,https://www.104.com.tw/company/wftqhi8,1001001001,電腦系統整合服務業,新北市,三重區,...,,待遇面議,全職,不需負擔管理責任,無需出差外派,日班,依公司規定,不限,不限,- 福利制度：泰偉電子提供具市場競爭力的薪資福利以及各項福利措施，以延攬各領域專業及深具潛力...
12465799,2024/02/06,《台中》C# 後端工程師,https://www.104.com.tw/job/7f6o7,70613972000,泰偉電子股份有限公司,https://www.104.com.tw/company/wftqhi8,1001001001,電腦系統整合服務業,台中市,北屯區,...,,待遇面議,全職,不需負擔管理責任,無需出差外派,日班,依公司規定,不限,不限,- 福利制度：泰偉電子提供具市場競爭力的薪資福利以及各項福利措施，以延攬各領域專業及深具潛力...


In [10]:
# create db
import mysql.connector

def createDB():
    db = mysql.connector.connect(
        host="localhost",
        user = "root",
        password = "Sql@1031",
        port = 3306
    )
    db_name = "job_db"
    cursor = db.cursor()
    cursor.execute(f"CREATE DATABASE IF NOT EXISTS {db_name}")
    cursor.execute("SHOW DATABASES")
    
    for x in cursor:
      print(x)

    db.commit()
    db.close()

createDB()

('information_schema',)
('job_db',)
('mysql',)
('performance_schema',)
('sakila',)
('sys',)
('world',)


In [46]:
import os, time
from dotenv import load_dotenv
load_dotenv()
os.getenv("MODE")
from sqlalchemy import create_engine, text
from urllib.parse import quote

# 對密碼進行 URL 編碼
user = "root"
password = quote("Sql@1031")
host = "localhost"
port = 3306
db_name = "job_db"
# 構建 MySQL 數據庫連接 URL
connector_url = f"mysql+mysqlconnector://{user}:{password}@{host}:{port}/{db_name}"
# print(connector_url)
# 創建引擎object
engine = create_engine(connector_url, future = True)

# SQL 語句範例
# sql_statement = "SELECT * FROM your_table"  # 替換成實際的表格名稱
sql_job_table =  """
            CREATE TABLE IF NOT EXISTS JobsInfo  (
            `更新日期` DATE,
            `職缺名稱` VARCHAR(100),
            `公司名稱` VARCHAR(100),
            `工作內容` TEXT,
            `職務類別` INT,
            `工作待遇` VARCHAR(100),
            `工作性質` INT,
            `縣市` INT,
            `上班地點` VARCHAR(100),
            `管理責任` INT,
            `出差外派` INT,
            `上班時段` VARCHAR(100),
            `休假制度` INT,
            `可上班日` INT,
            `需求人數` VARCHAR(50),
            `工作經歷` INT,
            `學歷要求` INT,
            `科系要求` VARCHAR(50),
            `語文條件` VARCHAR(50),
            `擅長工具` VARCHAR(500),
            `工作技能` VARCHAR(500),
            `其他要求` TEXT,
            `連結` TEXT,
            PRIMARY KEY (`連結`(255))
);
        """

with engine.connect() as connection:
    print("test OK")
    connection.execute(text(sql_job_table))


test OK


# Step 5 - Data Application

# Step 99. Test Area - 測試區域

In [ ]:
# custom filter params for search - simon
# role = {'ro':'全職'}
# zone = {'zone': '4,5'}
# indcat = {'indcat':'1001000000,1002000000,1012000000,1014000000'}
# exclude = {'excludeJobKeyword': '軟體,會計,文件整理,行政,百貨,資訊,品管,財務,守衛,技術員,法務,品檢,登打,輪班,無經驗,客戶'}
# mode = {'mode':'列表'}  # 一次能呈現比較多筆資料
# order = {'order':'日期排序'}
# asc = {'asc':'遞減'}
# filter_params = get_filter_params(role, zone, indcat, exclude, mode, order, asc)

In [ ]:
# 如果你在一個 Python script 中執行這樣的程式碼，你可能需要使用 asyncio.run() 函數。
# 在異步程式碼中，如果你使用 asyncio.gather 函數收集異步任務的結果，而這些任務沒有顯式返回值，
# gather 函數將返回一個包含每個異步任務結果的列表，而這些結果通常是 None。

import asyncio

async def async_example(i):
    print(f"Start asynchronous task {i}")
    await asyncio.sleep(1)
    print(f"End asynchronous task {i}")
    return f"Result of task {i}"

# 直接在 Jupyter cell 中執行
tasks = [async_example(i) for i in range(5)]
results = await asyncio.gather(*tasks)
print(results)